# 获取文件对象

In [1]:
# 获取路径下所有文件或文件夹——生成器
import os

def get_file_name(dir_path:str):
    
    # 二阶文件夹数据获取
    file_list = os.listdir(dir_path)
    for dir in file_list:
        file_dir_path = os.path.join(dir_path,dir)
        file_name_list = os.listdir(file_dir_path)
        for num in range(len(file_name_list)):
            file_name = file_name_list[num]
            file_path = os.path.join(file_dir_path,file_name)
            yield file_path

# 语音长度判断

In [2]:
import librosa
import wave
import contextlib
import eyed3
from pydub import AudioSegment

def get_duration_mp3(file_path):
    """
    获取mp3音频文件时长
    :param file_path:
    :return:
    """
    mp3Info = eyed3.load(file_path)
    return mp3Info.info.time_secs


def get_duration_wav(file_path):
    """
    获取wav音频文件时长
    :param file_path:
    :return:
    """
    with contextlib.closing(wave.open(file_path, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
    return duration


def get_duration_mp3_and_wav(file_path):
     """
     获取mp3/wav音频文件时长
     :param file_path:
     :return:
     """
     duration = librosa.get_duration(filename=file_path)
     return duration
 

# 获取wav音频时长的又一种方式
def get_wav_make(file_path):
    sound = AudioSegment.from_wav(file_path)
    duration = sound.duration_seconds  # 音频时长（ms）
    return duration

file_genertor =  get_file_name(r'C:\Users\丁健一\Desktop\政府满意度项目\人工外呼相关\呼出结果分析\转译项目\项目文件\datasets\录音文件')
file_path = next(file_genertor)
file_path

# 创建转译文件夹

In [3]:
def dir_maker(dir_base:str,file_path:str):

        # 对于给的文件路径取出其基路径与文件名
        file_base_path, file_name = os.path.split(file_path)
        file_dir_name = file_base_path.split('\\')[-1]
        file_pure_name = file_name.split('.')[0]
        
        # 在指定的根目录下新建文件夹
        dir_base = os.path.abspath(dir_base)
        dir_path = os.path.join(dir_base,file_dir_name,file_pure_name)

        # 建立各个文件夹
        dir_list = ['mp3_to_wav','resampling','cut_record','to_text']
        all_new_dirs = [os.path.join(dir_path,name) for name in dir_list]
        try: 
            for dir in all_new_dirs:
                os.makedirs(dir,exist_ok=True)

        except Exception as e:
            pass

        # 返回一个列表，元素依次为文件路径，转换为wav的路径,重采样文件夹路径，切割录音路径，转译文件路径
        return [file_path] + all_new_dirs

file_path,wav_dir,resample_dir,cut_dir,text_dir = dir_maker(dir_base=r'C:\Users\丁健一\Desktop\政府满意度项目\人工外呼相关\呼出结果分析\转译项目\项目文件\datasets\转换文件',file_path = file_path)

# 转换文件格式_1

In [4]:
from pydub import AudioSegment
import wave
import io

AudioSegment.converter = r"C:\Program Files\ffmpeg\bin\ffmpeg.exe"
AudioSegment.ffmpeg = r"C:\Program Files\ffmpeg\bin\ffmpeg.exe"
AudioSegment.ffprobe = r"C:\Program Files\ffmpeg\bin\ffprobe.exe"



def mp3_to_wav(file_path, wav_dir):

    # 处理文件路径
    file_pure_name = os.path.split(file_path)[-1].split('.')[0]
    file_wav_name = f'{file_pure_name}' + '.wav'
    wav_path = os.path.join(wav_dir,file_wav_name)

    song = AudioSegment.from_mp3(file_path)
    song.export(wav_path, format="wav")
    return wav_path

In [5]:
# wav_path = mp3_to_wav(file_path,wav_dir)
# wav_path

# 转换文件格式

In [6]:
import os
from ffmpy import FFmpeg


# MP3转wav
def audio_transfor(audio_path: str, output_dir: str):
    ext = os.path.basename(audio_path).strip().split('.')[-1]
    if ext != 'mp3':
        raise Exception('format is not mp3')

    result = os.path.join(output_dir, '{}.{}'.format(os.path.basename(audio_path).strip().split('.')[0], 'wav'))
    filter_cmd = '-f wav -ac 1 -ar 16000'
    ff = FFmpeg(
        inputs={
            audio_path: None}, outputs={
            result: filter_cmd})
    # print(ff.cmd)
    ff.run()
    return result

# 重采样

In [7]:
# 批量转换采样率
import os
import librosa
import soundfile


# 定义转换采样率的函数，接收3个变量：原音频路径、重新采样后的音频存储路径、目标采样率
def change_sample_rate(path, new_dir_path, new_sample_rate):
    wavfile = os.path.split(path)[-1] # 提取音频文件名，如“1.wav"
    # path.split('/')[-1]  
    # new_file_name = wavfile.split('.')[0] + '_8k.wav'      #此行代码可用于对转换后的文件进行重命名（如有需要）

    signal, sr = librosa.load(path, sr=None)  # 调用librosa载入音频
    try:
        new_signal = librosa.resample(signal, sr, new_sample_rate)  # 调用librosa进行音频采样率转换
    except Exception as e:
        pass

    new_path = os.path.join(new_dir_path, wavfile)  # 指定输出音频的路径，音频文件与原音频同名
    # new_path = os.path.join(new_dir_path, new_file_name)      #若需要改名则启用此行代码
    # print(new_path)

    # librosa.output.write_wav(new_path, new_signal , new_sample_rate)      #因版本问题，此方法可能用不了
    soundfile.write(new_path, new_signal, new_sample_rate)
    return new_path

In [8]:
# resample_path = change_sample_rate(wav_path,resample_dir,16000)
# resample_path

# 剪切录音文件

In [9]:
# 引入auditok库
import auditok
import soundfile
import os


# 输入类别为audio
def qiefen(path,save_path, ty='audio', mmin_dur=1, mmax_dur=100000, mmax_silence=1, menergy_threshold=55):
    audio_file = path
    audio, audio_sample_rate = soundfile.read(
        audio_file, dtype="int16", always_2d=True)

    audio_regions = auditok.split(
        audio_file,
        min_dur=mmin_dur,  # minimum duration of a valid audio event in seconds
        max_dur=mmax_dur,  # maximum duration of an event
        # maximum duration of tolerated continuous silence within an event
        max_silence=mmax_silence,
        energy_threshold=menergy_threshold  # threshold of detection
    )

    for i, r in enumerate(audio_regions):
        # Regions returned by `split` have 'start' and 'end' metadata fields
        # print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))

        num = i
        # 为了取前三位数字排序
        s = '000000' + str(num)

        file_name = s[-3:] + '-' + '{meta.start:.3f}-{meta.end:.3f}' + '.wav'
        file_save = os.path.join(save_path,file_name)

        filename = r.save(file_save)
        # print("region saved as: {}".format(filename))
    # print(f'完成转换，共计{num+1}个文件')

In [10]:
# qiefen(resample_path,cut_dir)

# 语音转译

In [11]:
from paddlespeech.cli.asr.infer import ASRExecutor
from paddlespeech.cli.text.infer import TextExecutor

In [12]:
asr = ASRExecutor()
text_punc = TextExecutor()

def record2text(file_path:str,cut_dir:str,text_dir:str):
    error_set = set()

    try:    
        file_pure_name = os.path.split(file_path)[-1].split('.')[0]
        file_name = file_pure_name + '.txt'

        text = ''
        wav_list = os.listdir(cut_dir)
        for wav in wav_list:
            wav_path = os.path.join(cut_dir,wav)
            time_long = get_duration_mp3_and_wav(wav_path)
            if time_long < 50:
                text += asr(audio_file=wav_path)
            else:
                error_set.add(file_pure_name)
        result = text_punc(text=text)

        if text != '':
            with open(os.path.join(text_dir,file_name),'w+') as f:
                f.write(result)
        else:
            error_set.add(file_pure_name)

    except Exception as e:
        print(f'{file_name}文件转译过程中出现问题。')
        pass

    finally:
        if error_set != []:
            return list(error_set)
        else:
            return []

In [13]:
# record2text(file_path,cut_dir,text_dir)

In [14]:
import time

def main(dir_path:str,dir_to:str):
    error_num = []

    file_all = get_file_name(dir_path)
    num = 0 # 计数
    while True:
        try:
            file_path = next(file_all)
            duration = get_duration_mp3_and_wav(file_path)
            if duration >= 180:
                num += 1
                file_path,wav_dir,resample_dir,cut_dir,text_dir \
                    = dir_maker(dir_base=dir_to,file_path = file_path)
                wav_path = audio_transfor(file_path,wav_dir)
                #print('完成wav格式转换')
                resample_path = change_sample_rate(wav_path,resample_dir,16000)
                #print('完成重采样')
                qiefen(resample_path,cut_dir)
                #print('完成切分录音')
                error_list = record2text(file_path,cut_dir,text_dir)
                error_num += error_list
                #print('完成文本转换')
                #print(f'已完成{num}条')
                if num%10 == 0:
                    print(f'已完成{num}条')
            else:
                pass
        except StopIteration as e:
            break

    if error_num != []:
        with open('./log/log.txt','a+') as f:
            f.write(str(error_num))

In [15]:
file_all_dir = r'C:\Users\丁健一\Desktop\政府满意度项目\人工外呼相关\呼出结果分析\转译项目\项目文件\datasets\测试'
change_base=r'C:\Users\丁健一\Desktop\政府满意度项目\人工外呼相关\呼出结果分析\转译项目\项目文件\datasets\转换文件'

In [16]:
import warnings
warnings.filterwarnings('ignore')

main(dir_path=file_all_dir,dir_to=change_base)

2022-11-15 13:44:58.299 | INFO     | paddlespeech.s2t.modules.ctc:<module>:45 - paddlespeech_ctcdecoders not installed!
2022-11-15 13:44:58.585 | INFO     | paddlespeech.s2t.modules.embedding:__init__:153 - max len: 5000


[2022-11-15 13:46:23,588] [    INFO] - Already cached C:\Users\丁健一\.paddlenlp\models\ernie-1.0\vocab.txt


人大代表_蔡高旭_20221103_20时40分05秒_15777339996.txt文件转译过程中出现问题。
政务服务对象_吴海云_20221105_15时24分29秒_18290104229.txt文件转译过程中出现问题。
政务服务对象_王玉兰_20221105_15时16分36秒_13217836272.txt文件转译过程中出现问题。
政务服务对象_蔡常爱_20221103_20时29分06秒_13560186546.txt文件转译过程中出现问题。
政务服务对象_谢丽芳_20221105_15时32分03秒_15277380324.txt文件转译过程中出现问题。
政务服务对象_韦建慧_20221104_15时46分41秒_15078977984.txt文件转译过程中出现问题。
政务服务对象_黄从超_20221104_12时28分26秒_18877305822.txt文件转译过程中出现问题。
政务服务对象_黄荣福_20221103_17时21分02秒_18290157068.txt文件转译过程中出现问题。
热线投诉群众(部门)_附件4热线13504011829_20221106_14时52分07秒_13504011829.txt文件转译过程中出现问题。
已完成10条
热线投诉群众(部门)_附件4热线13597307799_20221106_15时09分12秒_13597307799.txt文件转译过程中出现问题。
热线投诉群众(部门)_附件4热线13978355827_20221106_15时27分23秒_13978355827.txt文件转译过程中出现问题。
热线投诉群众(部门)_附件4热线15394427231_20221108_14时49分41秒_15394427231.txt文件转译过程中出现问题。
热线投诉群众(部门)_附件4热线15507730155_20221106_15时44分13秒_15507730155.txt文件转译过程中出现问题。
热线投诉群众(部门)_附件4热线15577335116_20221106_16时01分24秒_15577335116.txt文件转译过程中出现问题。
热线投诉群众(部门)_附件4热线18607737523_20221108_14时38分19秒_18607737523.txt文件转译过程中出现问题。
热线投诉群众